In [1]:
import torch
from matplotlib import pyplot as plt
from sklearn.datasets import load_iris
import numpy as np
from torchvision import transforms
from torchvision.datasets import FashionMNIST
%matplotlib inline

In [2]:
import os
os.environ['http.proxy'] = 'http://yikuni.com:13128'
os.environ['https.proxy'] = 'http://yikuni.com:13128'

In [3]:
trans = transforms.ToTensor()
mnist_train = FashionMNIST(
    root="./data",
    train=True,
    transform=trans,
    download=True
)
mnist_test = FashionMNIST(
    root="./data",
    train=False,
    transform=trans,
    download=True
)

Using downloaded and verified file: ./data\FashionMNIST\raw\train-images-idx3-ubyte.gz
Extracting ./data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data\FashionMNIST\raw

Using downloaded and verified file: ./data\FashionMNIST\raw\train-labels-idx1-ubyte.gz
Extracting ./data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw

Using downloaded and verified file: ./data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz
Extracting ./data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5148/5148 [00:00<?, ?it/s]

Extracting ./data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



In [5]:
len(mnist_train), len(mnist_test)

(60000, 10000)

In [8]:
train_dataloader = torch.utils.data.DataLoader(mnist_train, batch_size=64, shuffle=True, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(mnist_test, batch_size=64, shuffle=True, num_workers=4)

In [13]:
def softmax(X):
    X_exp = torch.exp(X)
    return X_exp / X_exp.sum(1, keepdim=True) 

In [21]:
X = torch.normal(0, 1, (3, 4))
X_prob = softmax(X)
X_prob, X_prob.sum(1)

(tensor([[0.3584, 0.4386, 0.0239, 0.1791],
         [0.2746, 0.2152, 0.1982, 0.3119],
         [0.0268, 0.1300, 0.8130, 0.0301]]),
 tensor([1., 1., 1.]))

In [17]:
num_inputs = 4
num_outputs = 3
W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

In [20]:
def softmax_forward(X):
    global W, b
    return softmax(torch.matmul(X, W) + b)

In [22]:
softmax_forward(X)

tensor([[0.3279, 0.3414, 0.3307],
        [0.3321, 0.3357, 0.3322],
        [0.3411, 0.3249, 0.3340]], grad_fn=<DivBackward0>)

In [23]:
def cross_entropy(y_hat, y):
    return - torch.log(y_hat[range(len(y_hat)), y])

In [ ]:
loss_history = []
for epoch in range(100):
    for X, y in train_dataloader:
        y_hat = softmax_forward(X)
        loss = cross_entropy(y_hat, y)
        x